In [1]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession, Row, Column, SQLContext, DataFrame, DataFrameStatFunctions
from pyspark.sql.functions import split
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("app1").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DoubleType

import calendar
from datetime import date,datetime
import math
from pyspark.sql.functions import date_format, lit

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pyarrow.parquet as pq
import scipy.io
import pandas as pd
le = LabelEncoder()

from sklearn.feature_selection import VarianceThreshold
from skfeature.function.similarity_based import fisher_score
from skfeature.utility import construct_W

from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

import time

In [104]:
results = sqlContext.read.format('csv').options(header='true',delimiter=';',mode='DROPMALFORMED',inferSchema=True).load("results/cardio/cardioDTK1.csv")

In [105]:
results.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- classifier: string (nullable = true)
 |-- resampler: string (nullable = true)
 |-- selection_algorithm: string (nullable = true)
 |-- accuracy: double (nullable = true)
 |-- precision: double (nullable = true)
 |-- recall: double (nullable = true)
 |-- duration: double (nullable = true)



In [106]:
part = results.select('resampler','accuracy','precision','recall').where(results.resampler.contains('SMOTE'))

In [107]:
part.describe().show()

+-------+---------+--------------------+-------------------+-------------------+
|summary|resampler|            accuracy|          precision|             recall|
+-------+---------+--------------------+-------------------+-------------------+
|  count|       32|                  32|                 32|                 32|
|   mean|     null|   0.912669234360411| 0.7794489112754441| 0.7366606125997581|
| stddev|     null|0.056335273075582025|0.17264722912459002|0.20326718116754594|
|    min|     none|  0.8590102707749766| 0.6120222409120643| 0.5380952380952381|
|    max|     none|  0.9897292250233427| 0.9894179894179894| 0.9767165943239519|
+-------+---------+--------------------+-------------------+-------------------+

